In [1]:
#Basic imports
import pandas as pd
import numpy as np
import pickle
from collections import defaultdict
from os import environ, getcwd
import sys

#Classifier imports
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, RandomForestRegressor, BaggingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from xgboost import XGBClassifier

#ML framework imports
from sklearn.metrics import auc, roc_auc_score, precision_recall_curve
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import scale

#Downsamplers imports - prototype generation
from imblearn.under_sampling import ClusterCentroids

#Downsamplers imports - prototype selection - controlled
from imblearn.under_sampling import RandomUnderSampler, NearMiss

#Downsamplers imports - prototype selection - Cleaning techniques
from imblearn.under_sampling import TomekLinks, EditedNearestNeighbours, RepeatedEditedNearestNeighbours

#Downsamplers imports - prototype selection - Cleaning techniques - Condensed nearest neighbors and derived algorithms
from imblearn.under_sampling import CondensedNearestNeighbour, OneSidedSelection, NeighbourhoodCleaningRule

#Downsamplers imports - prototype selection - Cleaning techniques
from imblearn.under_sampling import InstanceHardnessThreshold

#Import utils functions
curr_dir = !pwd
sys.path.append(curr_dir[0]+"/utils")
from neg_pos_funcs import create_negatives_datasets, create_positives_datasets
from CV_funcs import add_domain_name_from_table_idx, calc_CV_idx_iterative

from IPython.core.display import HTML
HTML("<style>.container { width:100% !important; }</style>")

ABSOLUTE_NEGATIVES = False
FILTER_DOMAIN = False
FILTER_MAX_SCORE_ZERO = False
out_dir = "mediode_NegLigand_NoFilter"

### Reading the input dataset

In [2]:
curr_dir = !pwd
input_path = curr_dir[0]+"/domains_similarity/filtered_features_table/"
filename = "windowed_positions_features_mediode_filter_01.25.18.csv"

#input_path = curr_dir[0]+"/../9.Features_exploration/binding_df/10/"
#filename = "positions_features_01.25.18.csv"

bind_scores_num = 10
ligands = ["dna", "dnabase", "dnabackbone", "rna", "rnabase", "rnabackbone", "peptide", "ion", "metabolite"]

#Features table
features_all = pd.read_csv(input_path+filename, sep='\t', index_col=0)
#Features columns names, without the labels (the binding scores)
features_cols = features_all.columns.tolist()
#removing binding scores and domain name
for ligand in ligands:
    score_str = ligand+"_binding_score"
    features_cols.remove(score_str)
features_cols.remove("max_binding_score")
features_cols.remove("domain_name")


print "all samples positions #: "+str(features_all.shape[0])

#lignd binding domains dictionary
with open(curr_dir[0]+"/ligands_negatives_domains_dict.pik", 'rb') as handle:
        negatives_dict = pickle.load(handle)

#CV splits dictionary
with open(curr_dir[0]+"/CV_splits/domain_10_splits_dict.pik", 'rb') as handle:
        splits_dict = pickle.load(handle)

all samples positions #: 38944


#### Dataset of negative examples

In [3]:
ligands_negatives_df = create_negatives_datasets(FILTER_DOMAIN, ABSOLUTE_NEGATIVES, FILTER_MAX_SCORE_ZERO, features_all, features_cols)

dna non-binding #:38095
dnabase non-binding #:38577
dnabackbone non-binding #:38203
rna non-binding #:38047
rnabase non-binding #:38407
rnabackbone non-binding #:38223
peptide non-binding #:35437
ion non-binding #:34488
metabolite non-binding #:33971
all_ligands non-binding #:27191


#### Datasets of positive examples by ligand

In [4]:
bind_th = 0.1
ligands_features_df = create_positives_datasets(bind_th, features_all, features_cols)

dna #: 501
dnabase #: 193
dnabackbone #: 408
rna #: 433
rnabase #: 224
rnabackbone #: 308
peptide #: 1496
ion #: 1093
metabolite #: 1525


#### Dataset of positive examples - all ligands combined

In [5]:
all_ligands_features_df = pd.concat([ligands_features_df["dna"], ligands_features_df["dnabase"], ligands_features_df["dnabackbone"], ligands_features_df["rna"], ligands_features_df["rnabase"], 
                                     ligands_features_df["rnabackbone"], ligands_features_df["ion"], ligands_features_df["peptide"], ligands_features_df["metabolite"]])
all_ligands_features_df = all_ligands_features_df.drop_duplicates()
print "all_ligands #: "+str(all_ligands_features_df.shape[0])
ligands_features_df["all_ligands"] = all_ligands_features_df

all_ligands #: 4518


### Reading env input for downsampler technique, ligand and classifier

In [6]:
#Reading the ligand input
try:
    ligand = environ['ligand']
except:
    ligand = "dna"
print "ligand = "+ligand
    
#Reading the downsampler input
try: 
    downsample_method = environ['down']
except:
    downsample_method = "NoDown"
print "downsample_method = "+downsample_method

#Reading the classifier input
try: 
    classifier_method = environ['classifier']
except:
    classifier_method = "XGB"
print "classifier_method = "+classifier_method

ligand = dna
downsample_method = NoDown
classifier_method = XGB


### Models tested (and their hyper-parameters)

In [7]:
classifiers = {}
classifiers["Logistic"] = LogisticRegression(C=0.001, random_state=0)
classifiers["RF"] = RandomForestClassifier(n_estimators=1000, n_jobs=-1, random_state=0)  
#classifiers["RF"] = RandomForestRegressor(n_estimators=1000)  
classifiers["KNN"] = KNeighborsClassifier(n_neighbors=100, n_jobs=-1)
#classifiers["KNN"] = KNeighborsRegressor(n_neighbors=100)
classifiers["SVM"] = SVC(kernel='rbf', probability=True, random_state=0)
classifiers["ADA"] = AdaBoostClassifier(n_estimators=1000, random_state=0)
# classifiers["ADA-Log"] = AdaBoostClassifier(base_estimator=classifiers["Logistic"], n_estimators=1000, random_state=0)
# classifiers["Bag-Log"] = BaggingClassifier(base_estimator=classifiers["Logistic"], n_estimators=1000, n_jobs=-1, random_state=0)
#Init the XGB according to the ligands neg/pos ratio
ligand_pos = ligands_features_df[ligand].shape[0]
ligand_neg = ligands_negatives_df[ligand].shape[0]
scale_weight = ligand_neg/float(ligand_pos)
classifiers["XGB"] = XGBClassifier(n_estimators=1000, n_jobs=-1, random_state=0, max_depth=6, min_child_weight=0.05, colsample_bytree=0.5, scale_pos_weight=scale_weight)

### Downsamplers tested

In [8]:
#documentation on techniques: http://contrib.scikit-learn.org/imbalanced-learn/stable/under_sampling.html#cleaning-under-sampling-techniques
downsamplers = defaultdict(dict)

##Prototype generation##
downsamplers["ClusterCentroids"] = ClusterCentroids(random_state=0)

##Prototype selection##
#Contolled#
downsamplers["RandomUnderSampler"] = RandomUnderSampler(random_state=0)
downsamplers["NearMiss3"] = NearMiss(random_state=0, version=3)
downsamplers["NearMiss2"] = NearMiss(random_state=0, version=2)
downsamplers["NearMiss1"] = NearMiss(random_state=0, version=1)

#Cleaning#
downsamplers["TomekLinks"] = TomekLinks(random_state=0)
downsamplers["EditedNearestNeighbours"] = EditedNearestNeighbours(random_state=0)
downsamplers["RepeatedEditedNearestNeighbours"] = RepeatedEditedNearestNeighbours(random_state=0)
downsamplers["NeighbourhoodCleaningRule"] = NeighbourhoodCleaningRule(random_state=0)

# Instance hardness threshold#
downsamplers["InstanceHardnessThreshold"]["KNN"] = InstanceHardnessThreshold(random_state=0, estimator=classifiers["KNN"])
#downsamplers["InstanceHardnessThreshold"]["KNN"] = InstanceHardnessThreshold(random_state=0, estimator= KNeighborsClassifier(n_neighbors=100))
downsamplers["InstanceHardnessThreshold"]["SVM"] = InstanceHardnessThreshold(random_state=0, estimator=classifiers["SVM"])
downsamplers["InstanceHardnessThreshold"]["RF"] = InstanceHardnessThreshold(random_state=0, estimator=classifiers["RF"])
#downsamplers["InstanceHardnessThreshold"]["RF"] = InstanceHardnessThreshold(random_state=0, estimator=RandomForestClassifier(n_estimators=1000))
downsamplers["InstanceHardnessThreshold"]["Logistic"] = InstanceHardnessThreshold(random_state=0, estimator=classifiers["Logistic"])
downsamplers["InstanceHardnessThreshold"]["ADA"] = InstanceHardnessThreshold(random_state=0, estimator=classifiers["ADA"])
# downsamplers["InstanceHardnessThreshold"]["ADA-Log"] = InstanceHardnessThreshold(random_state=0, estimator=classifiers["ADA-Log"])
# downsamplers["InstanceHardnessThreshold"]["Bag-Log"] = InstanceHardnessThreshold(random_state=0, estimator=classifiers["Bag-Log"])

### Test model functions

In [9]:
def test_model_iterative_fixed(pred_dict, auc_dict, auprc_dict, ligand_bind_features, ligand_negatives_features, ligand_name, downsample_method, features=[]):
    """
    Test different models in 10-folds cross-validation.
    """
    
    #Default: Exclude no features
    if len(features) == 0:
        features = np.ones([ligand_bind_features.shape[1],]).astype(bool)
        
    #Arranging the features table by the CV order, for each model
    features_pred_dfs = dict.fromkeys(classifiers.keys())
    
    models_req_scaling = ["SVM", "KNN"]
    
    for classifier in classifiers.keys():
        classifier = classifier_method
        model = classifiers[classifier]
        features_pred_dfs[classifier] = pd.DataFrame()
        
        #Create X and y with included features
        X = pd.concat([ligand_bind_features.iloc[:,features], ligand_negatives_features.iloc[:,features]])
        
        if (classifier in models_req_scaling):
            idx = X.index
            cols = X.columns
            X = pd.DataFrame(scale(X)) #Is z-scoring the data needed?
            X.index = idx #Restoring indices after scaling
            X.columns = cols
        
        y = [1] * ligand_bind_features.shape[0]
        y.extend([0] * ligand_negatives_features.shape[0])
        y = np.array(y)
        y_df = pd.DataFrame(y)
        y_df.index = X.index
        y_df.columns = ["label"]

        cv_idx = calc_CV_idx_iterative(X, splits_dict)

        for k in range(len(cv_idx)):
            pred_idx = k+1
            print "fold #: "+str(pred_idx)
            test_index = cv_idx[k]["test"]
            train_index = cv_idx[k]["train"]
            X_train, X_test = X.loc[train_index,:], X.loc[test_index,:]
            y_train, y_test = y_df.loc[train_index,:], y_df.loc[test_index,:]

            #Down-sample negative examples to have balanced classes
            if (downsample_method == "NoDown"):
                X_train_sampled = X_train
                y_train_sampled = y_train
            else:
                if (downsample_method == "InstanceHardnessThreshold"):
                    downsampler = downsamplers[downsample_method][classifier]
                else:
                    downsampler = downsamplers[downsample_method]

                X_train_sampled, y_train_sampled = downsampler.fit_sample(X_train, y_train["label"])
            
            #fit to training data
            model = classifiers[classifier]
            model.fit(X_train_sampled, y_train_sampled["label"])
            
            #probs = model.predict(X_test)
            #probs_list = probs
            probs_list = []
            probs = model.predict_proba(X_test)
            for l in probs:
                probs_list.append(l[1])
                
            pred_dict["obs"].extend(y_test["label"])
            pred_dict["prob"].extend(probs_list)
            fold_list = [pred_idx] * len(probs_list)
            pred_dict["fold"].extend(fold_list)

            model_list = [classifier] * len(probs_list)
            pred_dict["model"].extend(model_list)

            #Update auc auprc dictionaries
            auc_dict[classifier].append(roc_auc_score(y_test["label"], probs[:, 1]))
            precision, recall, _ = precision_recall_curve(y_test["label"], probs[:, 1])
            
            #auc_dict[classifier].append(roc_auc_score(y_test, probs))
            #precision, recall, _ = precision_recall_curve(y_test, probs)
            
            auprc_dict[classifier].append(auc(recall, precision))
            
            #Update features table
            features_pred_dfs[classifier] = features_pred_dfs[classifier].append(X_test)
            pred_idx += 1
            
            print "AUC = "+str(auc_dict[classifier][-1])
            print "AUPRC = "+str(auprc_dict[classifier][-1])

        avg_auc = np.sum(auc_dict[classifier])/len(cv_idx)
        print "avg auc = "+str(avg_auc)
        print "auc var = "+str(np.var(auc_dict[classifier]))
        
        avg_auprc = np.sum(auprc_dict[classifier])/len(cv_idx)
        print "avg auprc = "+str(avg_auprc)
        print "auprc var = "+str(np.var(auprc_dict[classifier]))
            
        print "Finished "+ligand+" "+classifier
        break
    
    return features_pred_dfs

In [10]:
def test_model_stratified_per_ligand(pred_dict, auc_dict, auprc_dict, ligand_bind_features, ligand_negatives_features, ligand_name, downsample_method, features=[]):
    """
    Test different models in 10-folds cross-validation.
    """
    
    #Default: Exclude no features
    if len(features) == 0:
        features = np.ones([ligand_bind_features.shape[1],]).astype(bool)
        
    #Arranging the features table by the CV order, for each model
    features_pred_dfs = dict.fromkeys(classifiers.keys())
    
    models_req_scaling = ["SVM", "KNN"]
    
    for classifier in classifiers.keys():
        classifier = classifier_method
        model = classifiers[classifier]
        features_pred_dfs[classifier] = pd.DataFrame()
        
        #Create X and y with included features
        X = pd.concat([ligand_bind_features.iloc[:,features], ligand_negatives_features.iloc[:,features]])
        
        if (classifier in models_req_scaling):
            idx = X.index
            cols = X.columns
            X = pd.DataFrame(scale(X)) #Is z-scoring the data needed?
            X.index = idx #Restoring indices after scaling
            X.columns = cols

        y = [1] * ligand_bind_features.shape[0]
        y.extend([0] * ligand_negatives_features.shape[0])
        y = np.array(y)

        binding_skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=0)
        pred_idx = 1

        for train_index, test_index in binding_skf.split(X, y):
            print "fold #: "+str(pred_idx)
            X_train, X_test = X.iloc[train_index,:], X.iloc[test_index,:]
            y_train, y_test = y[train_index], y[test_index]

            #Down-sample negative examples to have balanced classes
            if (downsample_method == "NoDown"):
                X_train_sampled = X_train
                y_train_sampled = y_train
            else:
                if (downsample_method == "InstanceHardnessThreshold"):
                    downsampler = downsamplers[downsample_method][classifier]
                else:
                    downsampler = downsamplers[downsample_method]

                X_train_sampled, y_train_sampled = downsampler.fit_sample(X_train, y_train)
            
            #fit to training data
            model = classifiers[classifier]
            model.fit(X_train_sampled, y_train_sampled)
    

            #probs = model.predict(X_test)
            #probs_list = probs
            probs_list = []
            probs = model.predict_proba(X_test)
            for l in probs:
                probs_list.append(l[1])
                
            pred_dict["obs"].extend(y_test)
            pred_dict["prob"].extend(probs_list)
            fold_list = [pred_idx] * len(probs_list)
            pred_dict["fold"].extend(fold_list)

            model_list = [classifier] * len(probs_list)
            pred_dict["model"].extend(model_list)

            #Update auc auprc dictionaries
            auc_dict[classifier].append(roc_auc_score(y_test, probs[:, 1]))
            precision, recall, _ = precision_recall_curve(y_test, probs[:, 1])
            
            #auc_dict[classifier].append(roc_auc_score(y_test, probs))
            #precision, recall, _ = precision_recall_curve(y_test, probs)
            
            auprc_dict[classifier].append(auc(recall, precision))
            
            #Update features table
            features_pred_dfs[classifier] = features_pred_dfs[classifier].append(X_test)
            pred_idx += 1
            
            print "AUC = "+str(auc_dict[classifier][-1])
            print "AUPRC = "+str(auprc_dict[classifier][-1])

        avg_auc = np.sum(auc_dict[classifier])/10.0
        print "avg auc = "+str(avg_auc)
        
        avg_auprc = np.sum(auprc_dict[classifier])/10.0
        print "avg auprc = "+str(avg_auprc)
            
        print "Finished "+ligand+" "+classifier
        break
    
    return features_pred_dfs

In [11]:
def combine_features_predictions(ligand, ordered_features, pred_df):
    
    pred_res = pred_df.copy(deep=True)
    for classifier in classifiers.keys():
        classifier = classifier_method
        model_pred = pred_res[pred_res["model"] == classifier]
        model_pred.index = ordered_features[classifier].index
        
        #Creating the combined table
        features_pred = pd.concat([ordered_features[classifier], model_pred], axis=1)
        
        #Saving
        features_pred.to_csv(curr_dir[0]+"/pred_AUC_AUPRC/"+out_dir+"/"+downsample_method+"/01.25.2018_domain_CV/features_pred_tables/"+ligand+"_"+classifier+"_features_pred.csv", sep='\t')
        
        break

#### Predict for each ligand seperatelly

In [12]:
%%time
    
ligand_pos = ligands_features_df[ligand].shape[0]
ligand_neg = ligands_negatives_df[ligand].shape[0]
scale_weight = ligand_neg/float(ligand_pos)
classifiers["XGB"] = XGBClassifier(n_estimators=1000, n_jobs=-1, random_state=0, max_depth=6, min_child_weight=0.05, colsample_bytree=0.5, scale_pos_weight=scale_weight)

#Initialize dictionary
pred_dict = defaultdict(list)
auc_dict = defaultdict(list)
auprc_dict = defaultdict(list)

ordered_features = test_model_iterative_fixed(pred_dict, auc_dict, auprc_dict, ligands_features_df[ligand], ligands_negatives_df[ligand], ligand, downsample_method)

pred_df = pd.DataFrame.from_dict(pred_dict)
auc_df = pd.DataFrame.from_dict(auc_dict)
auprc_df = pd.DataFrame.from_dict(auprc_dict)

#Save to file
pred_df.to_csv(curr_dir[0]+"/pred_AUC_AUPRC/"+out_dir+"/"+downsample_method+"/01.25.2018_domain_CV/"+ligand+"_"+classifier_method+"_10w.csv", sep='\t')
auc_df.to_csv(curr_dir[0]+"/pred_AUC_AUPRC/"+out_dir+"/"+downsample_method+"/01.25.2018_domain_CV/"+ligand+"_"+classifier_method+"_10w_auc.csv", sep='\t')
auprc_df.to_csv(curr_dir[0]+"/pred_AUC_AUPRC/"+out_dir+"/"+downsample_method+"/01.25.2018_domain_CV/"+ligand+"_"+classifier_method+"_10w_auprc.csv", sep='\t')

#Combine features and pred results to a unified table
#combine_features_predictions(ligand, ordered_features, pred_df)

print "Finished ligand "+ligand

fold #: 1
AUC = 0.850064837905
AUPRC = 0.560776630586
fold #: 2


KeyboardInterrupt: 